# ANITA API Guide

## I. Introduction

- Overview of ANITA and its purpose: ANITA (Advanced National ID and Transaction Assistant) is designed to facilitate secure and efficient interactions with smart cards, NFC tags, and related technologies. This API provides a comprehensive interface for developers to integrate ANITA's functionalities into their applications.
- Target audience for this guide: This guide is intended for software developers, system integrators, and technical professionals who wish to utilize ANITA's API to build applications that interact with smart cards, NFC devices, and related systems. Familiarity with RESTful API principles and basic programming concepts is recommended.
- API conventions:
  - RESTful principles: The ANITA API adheres to RESTful architectural principles, utilizing standard HTTP methods (GET, POST) for resource manipulation.
  - Data formats: API requests and responses primarily use JSON (JavaScript Object Notation) for data exchange.
  - Error handling: Standard HTTP status codes are used to indicate the success or failure of API requests. Detailed error messages are provided in the response body when errors occur.
- Base URL for the API: `https://api.anita.example.com/v1` (Note: This is a placeholder URL and should be replaced with the actual base URL).

## II. Authentication and Authorization

- Authentication methods:
  - API Keys: API keys are used to authenticate requests. Each user or application is assigned a unique API key, which must be included in the `X-API-Key` header of each request.
  - JWT (JSON Web Tokens): JWTs may be used for more advanced authentication scenarios. Clients can obtain a JWT by authenticating with a username and password, and then use the JWT in the `Authorization` header of subsequent requests.
- Authorization levels and roles:
  - The API supports different authorization levels and roles to control access to specific resources and functionalities. Common roles include:
    - `reader`: Read-only access to reader information.
    - `card_operator`: Ability to perform smart card operations.
    - `nfc_operator`: Ability to perform NFC operations.
    - `admin`: Full access to all API endpoints and functionalities.
- Example of authentication flow (API Key):

    ```http
    GET /api/readers HTTP/1.1
    X-API-Key: YOUR_API_KEY
    ```

    Example of authentication flow (JWT):

    1. Obtain JWT:

    ```http
    POST /api/auth/login HTTP/1.1
    Content-Type: application/json

    {
    "username": "your_username",
    "password": "your_password"
    }
    ```

    Response:

    ```json
    {
    "token": "YOUR_JWT_TOKEN"
    }
    ```

    1. Use JWT in subsequent requests:

    ```http
    GET /api/readers HTTP/1.1
    Authorization: Bearer YOUR_JWT_TOKEN
    ```

## III. API Endpoints

### A. Reader Management

- `/readers`
  - `GET`: List available readers
  - Description: Retrieves a list of available smart card readers connected to the system.
  - Request:

    ```http
    GET /api/readers HTTP/1.1
    X-API-Key: YOUR_API_KEY
    ```

  - Response format: JSON array of reader objects.

    ```json
    [
    {
    "name": "Reader 1",
    "type": "Smart Card Reader",
    "isContactless": true,
    "isContact": true
    },
    {
    "name": "Reader 2",
    "type": "NFC Reader",
    "isContactless": true,
    "isContact": false
    }
    ]
    ```

- `/smartcard/select`
  - `POST`: Select a specific reader
  - Description: Selects a specific smart card reader for subsequent operations.
  - Request:

    ```http
    POST /api/smartcard/select HTTP/1.1
    Content-Type: application/json
    X-API-Key: YOUR_API_KEY

    {
    "reader": "Reader 1"
    }
    ```

  - Request body: `{ reader: string }`
  - Response format: Success/error status.

    ```json
    {
    "status": "success",
    "message": "Reader 'Reader 1' selected."
    }
    ```

- `/smartcard/detect`
  - `GET`: Detect card status on the selected reader
  - Description: Detects the presence and status of a smart card in the selected reader.
  - Request:

    ```http
    GET /api/smartcard/detect HTTP/1.1
    X-API-Key: YOUR_API_KEY
    ```

  - Response format: Card presence, ATR, etc.

    ```json
    {
    "cardPresent": true,
    "atr": "3B6E00...",
    "protocol": "T=0"
    }
    ```

### B. Smartcard Operations

- `/smartcard/atr`
  - `GET`: Get the ATR (Answer To Reset) of the inserted smartcard
  - Description: Retrieves the ATR (Answer To Reset) of the smart card inserted in the selected reader.
  - Request:

    ```http
    GET /api/smartcard/atr HTTP/1.1
    X-API-Key: YOUR_API_KEY
    ```

  - Response format: ATR data.

    ```json
    {
    "atr": "3B6E00..."
    }
    ```

- `/smartcard/transmit`
  - `POST`: Transmit an APDU (Application Protocol Data Unit) command
  - Description: Transmits an APDU command to the smart card in the selected reader and returns the response.
  - Request:

    ```http
    POST /api/smartcard/transmit HTTP/1.1
    Content-Type: application/json
    X-API-Key: YOUR_API_KEY

    {
    "apdu": "00A404000E325041592E5359532E4444463031"
    }
    ```

  - Request body: `{ apdu: string }`
  - Response format: APDU response data.

    ```json
    {
    "response": "611E4F07A0000004101001500B54657374204170706C69636174696F6E870A0A023800950000000000009000"
    }
    ```

### C. NFC Operations

- `/nfc/read`
  - `GET`: Read NFC tag data
  - Description: Reads data from an NFC tag.
  - Request:

    ```http
    GET /api/nfc/read HTTP/1.1
    X-API-Key: YOUR_API_KEY
    ```

  - Response format: Tag type, UID, size, tech, result.

    ```json
    {
    "tagType": "Mifare Classic",
    "uid": "04B3982E432F80",
    "size": 4096,
    "tech": "IsoDep, MifareClassic",
    "result": "OK",
    "data": "..."
    }
    ```

- `/nfc/write` (if implemented)
  - `POST`: Write data to an NFC tag
  - Description: Writes data to an NFC tag.
  - Request:

    ```http
    POST /api/nfc/write HTTP/1.1
    Content-Type: application/json
    X-API-Key: YOUR_API_KEY

    {
    "data": "Your data to write"
    }
    ```

  - Request body: Data to write
  - Response format: Success/error status.

    ```json
    {
    "status": "success",
    "message": "Data written successfully."
    }
    ```

- `/nfc/clear`
  - `POST`: Clear data from an NFC tag
  - Description: Clears data from an NFC tag.
  - Request:

    ```http
    POST /api/nfc/clear HTTP/1.1
    Content-Type: application/json
    X-API-Key: YOUR_API_KEY

    {
    "reader": "NFC Reader"
    }
    ```

  - Request body: `{reader: reader}`
  - Response format: Success/error status.

    ```json
    {
    "status": "success",
    "message": "NFC tag cleared successfully."
    }
    ```

- `/nfc/write_raw`
  - `POST`: Write raw data to an NFC tag
  - Description: Writes raw data to an NFC tag, specifying the record type and payload.
  - Request:

    ```http
    POST /api/nfc/write_raw HTTP/1.1
    Content-Type: application/json
    X-API-Key: YOUR_API_KEY

    {
    "record_type": "text",
    "payload": "Hello, NFC!"
    }
    ```

  - Request body: `{ record_type: string, payload: string }`
  - Response format: Success/error status.

    ```json
    {
    "status": "success",
    "message": "Raw data written successfully."
    }
    ```

### D. MIFARE Operations (if implemented)

- `/mifare/read_sector`
  - `POST`: Read a specific sector from a MIFARE card
  - Description: Reads a specific sector from a MIFARE card.
  - Request:

    ```http
    POST /api/mifare/read_sector HTTP/1.1
    Content-Type: application/json
    X-API-Key: YOUR_API_KEY

    {
    "sector": 1
    }
    ```

  - Request body: Sector number
  - Response format: Sector data.

    ```json
    {
    "sector": 1,
    "data": "..."
    }
    ```

- `/mifare/write_block`
  - `POST`: Write data to a specific block on a MIFARE card
  - Description: Writes data to a specific block on a MIFARE card.
  - Request:

    ```http
    POST /api/mifare/write_block HTTP/1.1
    Content-Type: application/json
    X-API-Key: YOUR_API_KEY

    {
    "sector": 1,
    "block": 2,
    "data": "Your data"
    }
    ```

  - Request body: Sector, block number, data
  - Response format: Success/error status.

    ```json
    {
    "status": "success",
    "message": "Block written successfully."
    }
    ```

### E. Settings

- `/api/settings`
  - `GET`: Retrieve current settings
  - Description: Retrieves the current settings of the ANITA application.
  - Request:

    ```http
    GET /api/settings HTTP/1.1
    X-API-Key: YOUR_API_KEY
    ```

  - Response format: JSON object with settings (theme, logging, auto-refresh, etc.)

    ```json
    {
    "theme": "dark",
    "logging": true,
    "autoRefresh": false
    }
    ```

  - `POST`: Update settings
  - Description: Updates the settings of the ANITA application.
  - Request:

    ```http
    POST /api/settings HTTP/1.1
    Content-Type: application/json
    X-API-Key: YOUR_API_KEY

    {
    "theme": "light",
    "autoRefresh": true
    }
    ```

  - Request body: JSON object with settings to update
  - Response format: Success/error status

    ```json
    {
    "status": "success",
    "message": "Settings updated successfully."
    }
    ```

### F. System Information

- `/api/system/info`
  - `GET`: Retrieve system information (uptime, versions, etc.)
  - Description: Retrieves system information such as uptime, software versions, and hardware details.
  - Request:

    ```http
    GET /api/system/info HTTP/1.1
    X-API-Key: YOUR_API_KEY
    ```

  - Response format: JSON object with system information

    ```json
    {
    "uptime": "24 hours",
    "version": "1.0.0",
    "os": "Linux"
    }
    ```

### G. Simulation Mode

- `/toggle_simulation`
  - `POST`: Toggle simulation mode
  - Description: Toggles the simulation mode, allowing developers to test their applications without interacting with real hardware.
  - Request:

    ```http
    POST /api/toggle_simulation HTTP/1.1
    Content-Type: application/json
    X-API-Key: YOUR_API_KEY

    {
    "enabled": true
    }
    ```

  - Request body: `{ enabled: boolean }`
  - Response format: Success/error status

    ```json
    {
    "status": "success",
    "message": "Simulation mode enabled."
    }
    ```

## IV. Data Models

- Reader object:

    ```json
    {
    "name": "Reader 1",
    "type": "Smart Card Reader",
    "isContactless": true,
    "isContact": true
    }
    ```

- Smartcard data format:
  - ATR (Answer To Reset): A sequence of bytes returned by a smart card upon initial activation.
  - APDU (Application Protocol Data Unit): A command message sent to a smart card.

- NFC tag data format:
  - Tag Type: The type of NFC tag (e.g., Mifare Classic, NTAG213).
  - UID: The unique identifier of the NFC tag.
  - Data: The data stored on the NFC tag, which can be in various formats (e.g., NDEF).

- MIFARE sector/block data format:
  - Sector: A segment of memory on a MIFARE card.
  - Block: A smaller unit of memory within a sector.

- Settings data format:

    ```json
    {
    "theme": "dark",
    "logging": true,
    "autoRefresh": false
    }
    ```

- System information data format:

    ```json
    {
    "uptime": "24 hours",
    "version": "1.0.0",
    "os": "Linux"
    }
    ```

## V. Error Handling

- Common error codes and their meanings:
  - `400 Bad Request`: The request was malformed or invalid.
  - `401 Unauthorized`: Authentication failed or the user does not have permission to access the resource.
  - `403 Forbidden`: The user is not authorized to perform the requested action.
  - `404 Not Found`: The requested resource was not found.
  - `500 Internal Server Error`: An unexpected error occurred on the server.
- Example error responses:

    ```json
    {
    "error": "Unauthorized",
    "message": "Invalid API key."
    }
    ```

## VI. Rate Limiting (if implemented)

- Rate limits for API requests:
  - The API is rate-limited to prevent abuse and ensure fair usage. The rate limit is 100 requests per minute per API key.
- How to handle rate limit errors:
  - When the rate limit is exceeded, the API returns a `429 Too Many Requests` error. The response includes headers indicating the rate limit and the time until the limit resets.

    ```http
    HTTP/1.1 429 Too Many Requests
    Retry-After: 60
    X-RateLimit-Limit: 100
    X-RateLimit-Remaining: 0
    X-RateLimit-Reset: 60
    ```

  - Applications should handle this error by waiting until the rate limit resets before making further requests.

## VII. Examples

- Code examples in different languages (JavaScript, Python) demonstrating API usage.

  - JavaScript (Node.js):

    ```javascript
    const axios = require('axios');

    const apiKey = 'YOUR_API_KEY';
    const baseUrl = 'https://api.anita.example.com/v1';

    async function getReaders() {
    try {
    const response = await axios.get(`${baseUrl}/readers`, {
    headers: {
    'X-API-Key': apiKey
    }
    });
    console.log(response.data);
    } catch (error) {
    console.error(error.response.data);
    }
    }

    getReaders();
    ```

  - Python:

    ```python
    import requests

    api_key = 'YOUR_API_KEY'
    base_url = 'https://api.anita.example.com/v1'

    def get_readers():
    headers = {'X-API-Key': api_key}
    try:
    response = requests.get(f'{base_url}/readers', headers=headers)
    response.raise_for_status()
    print(response.json())
    except requests.exceptions.RequestException as e:
    print(e.response.json())

    get_readers()
    ```

## VIII. API Client Libraries (if available)

- Information about available client libraries for different languages (e.g., Python, Java, .NET).  Links to the libraries and documentation will be provided here.

## IX. Versioning

- API versioning strategy:
  - The API uses semantic versioning (e.g., v1, v2) to ensure backward compatibility.
- How to specify the API version in requests:
  - The API version is specified in the base URL (e.g., `https://api.anita.example.com/v1`).

## X. Support and Contact Information

- How to get support for the API:
  - Refer to the API documentation for detailed information and examples.
  - Check the FAQ section for answers to common questions.
- Contact information for the API developers:
  - Email: <support@anita.example.com>
  - Online form: <https://anita.example.com/support>